<a href="https://colab.research.google.com/github/betty6you/Jing_INFO5731_Spring2020/blob/main/In_class_exercise_08_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **The eighth in-class-exercise (20 points in total, 3/30/2021)**

The data for this exercise is from the dataset you created from assignment three. Please perform answer the following questions based on your data:

## (1) (10 points) Write a python program to extract the sentiment related terms from the corpus. You may use python package such as polyglot or external lexicon resources in the question. Rank the sentiment related terms by frequency.

In [56]:
!pip install polyglot
!pip install pyicu
!pip install pycld2
!pip install morfessor
!polyglot download sentiment2.en

import pandas as pd
data = pd.read_csv(r"https://raw.githubusercontent.com/betty6you/Jing_INFO5731_Spring2020/main/AmazonReviewupdated_sentiment_renew.csv")


from polyglot.text import Text # I tried the method offer online, but still it doesn't work, but the code in total should be correct
sentimentWords = []
for row_content in data['Review']:
  line_text = Text(row_content)
  for word in line_text.words:
    if word.polarity != 0:
      sentimentWords.append(word)
sentimentWords

[polyglot_data] Downloading package sentiment2.en to
[polyglot_data]     /root/polyglot_data...
[polyglot_data]   Package sentiment2.en is already up-to-date!


ImportError: ignored

## (2) (10 points) Compare the performance of the following tools in sentiment identification: TextBlob (https://textblob.readthedocs.io/en/dev/), VADER (https://github.com/cjhutto/vaderSentiment), TFIDF-based Support Vector Machine (SVM) (Split your data into training and testing data). Take your own annotation as the standard answers. 

Reference code: https://towardsdatascience.com/fine-grained-sentiment-analysis-in-python-part-1-2697bb111ed4

In [55]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')
import nltk
nltk.download('stopwords')
# TextBlob method
from textblob import TextBlob, Word, Blobber

data = pd.read_csv(r"https://raw.githubusercontent.com/betty6you/Jing_INFO5731_Spring2020/main/AmazonReviewupdated_sentiment_renew.csv")
print(data.shape)
data.head()
data['Polarity']=data['Review'].apply(lambda x: TextBlob(x).sentiment[0]) # if polarity is near 1, it's positive, if is near -1, it's negative
data['comp_senti'] = data['Polarity'].apply(lambda c: 'pos' if c >=0 else 'neg')
data

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
(111, 2)


,Review,sentiment,Polarity,comp_senti
0,product arrived damaged and not working seller...,negative,0.000000,pos
1,came in evening around 11pm so in thankful usp...,positive,0.240210,pos
2,this laptop wa ok for the price lack of usbc i...,negative,0.041667,pos
3,this is not the touch screen a advertised noth...,negative,-0.425000,neg
4,laptop randomly shut itself off and would not ...,negative,0.010606,pos
...,...,...,...,...
106,meet all my expectationsso far,positive,0.100000,pos
107,wasnt planning on buying a labtop but had to i...,positive,0.013300,pos
108,server it purpose for a field superintentent,positive,0.000000,pos
109,best laptop,positive,1.000000,pos


In [54]:
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

sid = SentimentIntensityAnalyzer()

import numpy as np
import pandas as pd

data = pd.read_csv(r"https://raw.githubusercontent.com/betty6you/Jing_INFO5731_Spring2020/main/AmazonReviewupdated_sentiment_renew.csv")
data.head()
data['scores'] = data['Review'].apply(lambda review: sid.polarity_scores(review))
data.head()
data['compound']  = data['scores'].apply(lambda score_dict: score_dict['compound'])
data.head()

data['comp_score'] = data['compound'].apply(lambda c: 'pos' if c >=0 else 'neg')

data.head()

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


,Review,sentiment,scores,compound,comp_score
0,product arrived damaged and not working seller...,negative,"{'neg': 0.289, 'neu': 0.652, 'pos': 0.059, 'co...",-0.8132,neg
1,came in evening around 11pm so in thankful usp...,positive,"{'neg': 0.046, 'neu': 0.72, 'pos': 0.234, 'com...",0.9676,pos
2,this laptop wa ok for the price lack of usbc i...,negative,"{'neg': 0.215, 'neu': 0.698, 'pos': 0.087, 'co...",-0.7591,neg
3,this is not the touch screen a advertised noth...,negative,"{'neg': 0.17, 'neu': 0.799, 'pos': 0.032, 'com...",-0.8147,neg
4,laptop randomly shut itself off and would not ...,negative,"{'neg': 0.038, 'neu': 0.85, 'pos': 0.112, 'com...",0.4881,pos


In [97]:
import pandas as pd
# train Data
trainData = pd.read_csv("https://raw.githubusercontent.com/betty6you/Jing_INFO5731_Spring2020/main/sentiment_Train_data.csv")
# test Data
testData = pd.read_csv("https://raw.githubusercontent.com/betty6you/Jing_INFO5731_Spring2020/main/sentiment_Test_data.csv")


from sklearn.feature_extraction.text import TfidfVectorizer
# Create feature vectors
vectorizer = TfidfVectorizer(min_df = 5,
                             max_df = 0.8,
                             sublinear_tf = True,
                             use_idf = True)
train_vectors = vectorizer.fit_transform(trainData['Review'])
test_vectors = vectorizer.transform(testData['Review'])

import time
from sklearn import svm
from sklearn.metrics import classification_report
# Perform classification with SVM, kernel=linear
classifier_linear = svm.SVC(kernel='linear')
t0 = time.time()
classifier_linear.fit(train_vectors, trainData['sentiment'])
t1 = time.time()
prediction_linear = classifier_linear.predict(test_vectors)
t2 = time.time()
time_linear_train = t1-t0
time_linear_predict = t2-t1

print("Training time: %fs; Prediction time: %fs" % (time_linear_train, time_linear_predict))
report = classification_report(testData['sentiment'], prediction_linear, output_dict=True)
print('positive: ', report['positive'])
print('negative: ', report['negative'])

Training time: 0.002400s; Prediction time: 0.000681s


KeyError: ignored